## The goal of this code is to plot spectra from the SDSS and the MMT, zooming in on lines of interest

### important relevant python packages

In [43]:
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from astropy.io import fits

### define function to get names for sdss spectra (from cb code)

In [44]:
def get_plot_title(filepath):
    fits_file = fits.open(filepath)

    file_char_list = list(filepath)
    spec_pos = filepath.find('spec-')
    title_list = file_char_list[spec_pos:-5]
    title = ''.join(title_list)
    headers = fits_file['PRIMARY'].header
    hh = str(int(headers['PLUG_RA'] * (24/360)))
    if not (len(hh) == 2):
        hh = '0' + hh
    mm = str(int((headers['PLUG_RA'] * (24/360) * 60) % 60))
    if not (len(mm) == 2):
        mm = '0' + mm
    ss = str(np.round((headers['PLUG_RA'] * (24/360) * 60 * 60) % 60, decimals=2))
    if not (len(str(int(np.round((headers['PLUG_RA'] * (24/360) * 60 * 60) % 60)))) == 2):
        ss = '0' + ss
    title = title + '/J' + hh + mm + ss
    return title

### identify relevant directory and fits files for sdss spectra

In [45]:
galfolpath = 'galaxies_fits_DR15/'
fitsfiles = [f for f in listdir(galfolpath)]
print(fitsfiles)

['spec-0913-52433-0300.fits', 'spec-1054-52516-0153.fits', 'spec-0986-52443-0580.fits', 'spec-0977-52410-0257.fits', 'spec-1305-52757-0191.fits', 'spec-1577-53495-0564.fits', 'spec-1627-53473-0154.fits', 'spec-1675-53466-0512.fits', 'spec-0326-52375-0382.fits', 'spec-6712-56421-0114.fits', 'spec-0483-51924-0628.fits', 'spec-0518-52282-0605.fits', 'spec-0566-52238-0319.fits', 'spec-0639-52146-0388.fits', 'spec-0581-52356-0196.fits', 'spec-0732-52221-0445.fits', 'spec-0761-54524-0409.fits']


### make a plot of sdss spectra

In [46]:
filename = 'sdss_specta.pdf'

with PdfPages(filename) as pdf:
    
    for i in range(1,len(fitsfiles)):
    
        # read in fits file
        hdu = fits.open(galfolpath+fitsfiles[i])

        coadd = hdu['COADD'].data
        fluxdata = coadd['flux']
        wavelength = 10 ** coadd['loglam']
        
        z = float(hdu[2].data['Z'])
    
        # plot the whole spectrum
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
    
        ax.plot(wavelength, fluxdata, linewidth=0.1)
    
        ax.set_xlim(3800, 9200)
        ax.set_ylim(0,20)
        ax.set_xlabel(r'Wavelength [$\AA$]', fontsize=13)
        ax.set_ylabel(r'$F_{\lambda}$ [$10^{-17}$ erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$]', fontsize=13)
        ax.set_title(get_plot_title(galfolpath+fitsfiles[i]))
    
        pdf.savefig()
        plt.close()
        
        # plot the [O II] line
        fig = plt.figure()
        ax = fig.add_subplot(4,2,1)
        
        o2 = 3727*(1+z)
        lo = o2-o2*1000/3e5
        hi = o2+o2*1000/3e5
        
        ax.plot(wavelength, fluxdata, linewidth=0.5)
        ax.set_xlim(lo, hi)
        ax.set_ylim(0,20)
        ax.set_title('[O II]')
        
        # plot the Hbeta line
        ax = fig.add_subplot(4,2,2)
        hb = 4861*(1+z)
        lo = hb-hb*1000/3e5
        hi = hb+hb*1000/3e5
        
        ax.plot(wavelength, fluxdata, linewidth=0.5)
        ax.set_xlim(lo, hi)
        ax.set_ylim(0,20)
        ax.set_title(r'H $\beta$')
        
        # plot the Mg II line
        ax = fig.add_subplot(4,2,5)
        mg2 = 2800*(1+z)
        lo = mg2-mg2*3000/3e5
        hi = mg2+mg2*3000/3e5
        
        ax.plot(wavelength, fluxdata, linewidth=0.5)
        ax.set_xlim(lo, hi)
        ax.set_ylim(0,20)
        ax.set_title('Mg II')   
        
        pdf.savefig()
        plt.close()

### identify relevant directory and fits files for mmt spectra

In [47]:
mmtgalfol = 'hizea/MMT/'
mmtfiles = [f for f in listdir(mmtgalfol)]
print(mmtfiles)

['J1125-0145_MMT.fit', 'J1450+4621_MMT.fit', 'J1219+0336_MMT.fit', 'J1506+5402_MMT.fit', 'J1613+2834_MMT.fit', 'J1558+3957_MMT.fit', 'J1713+2817_MMT.fit', 'J2116-0634_MMT.fit', 'J0826+4305_MMT.fit', 'J2140+1209_MMT.fit', 'J2118+0017_MMT.fit', 'J0901+0314_MMT.fit', 'J0905+5759_MMT.fit', 'J0944+0930_MMT.fit']


In [48]:
filename = 'mmt_specta.pdf'

with PdfPages(filename) as pdf:
    
    for i in range(0,len(mmtfiles)):
    
        hdu = fits.open(mmtgalfol+mmtfiles[i])

        maintable = hdu[1].data
        fluxdata = maintable['FLUX'][0]
        wavelength = maintable['WAVE'][0]
        
        name = maintable['SHORT_NAME'][0]
        
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
    
        ax.plot(wavelength, fluxdata)
    
        #ax.set_xlim(3800, 9200)
        #ax.set_ylim(0,20)
        ax.set_xlabel(r'Wavelength [$\AA$]', fontsize=13)
        ax.set_ylabel(r'$F_{\lambda}$ [$10^{-17}$ erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$]', fontsize=13)
        ax.set_title(name)
    
        pdf.savefig()
        plt.close()
        
        